## Identifying and resolving conflicts in a batch of VI files

ELGs - part 3 is sets 11 to 15

There are 750 visual inspections of a total of 250 unique objects
3.0 inspections per object.

81 conflicts, down to 49 if ignoring the low flag ones

In [141]:
49./250

0.196

In [1]:
save_filename='truth_table_ELGs_sets11_15.txt'

In [2]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

#import desispec
# library location will change ..
sys.path.append("/global/homes/r/rtojeiro/prospect/prospect/py")
from prospect import utils_specviewer,plotframes
import matplotlib.pyplot as plt 

In [3]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/'
tiles = ['67230']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']
obs_db = utils_specviewer.make_targetdict(tiledir, petals=petals, tiles=tiles) # tiles = optional 

In [4]:
pd.set_option('display.max_rows', 20)

In [5]:
#set to directory with all the VI files to merge
VI_dir = os.environ['HOME']+'/SV/VI_files/SV0/ELG/'

In [6]:
#read a set of files 
all_files = os.listdir(VI_dir)
vi_files=[]

for i in range(11,16):
    pattern = "desi*_"+str(i)+"_*.csv"
    print(pattern)
    for entry in all_files:
        if fnmatch.fnmatch(entry, pattern):
            vi_files.append(entry)

vi_files

desi*_11_*.csv
desi*_12_*.csv
desi*_13_*.csv
desi*_14_*.csv
desi*_15_*.csv


['desi-vi_SV0_ELG_tile67230_night20200315_11_BJW.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_11_CGS.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_11_VRK.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_12_DRP.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_12_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_12_VRK.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_13_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_13_CGS.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_13_DRP.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_14_DRP.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_14_BLH.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_14_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_15_TWL.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_15_DRP.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_15_BJW.csv']

In [7]:
vi = pd.read_csv(VI_dir + vi_files[0], delimiter = " , ", engine='python')

for i in range(1,len(vi_files)):
    print(vi_files[i])
    vi2 = pd.read_csv(VI_dir + vi_files[i], delimiter = " , ", engine='python')
    vi = vi.append(vi2, ignore_index=True)
    
#make groups of visual inspections, grouped by unique objects, and state number of single and multiple VIs
vi_gp = vi.groupby(['TargetID'])
print('There are ' + str(len(vi)) + ' visual inspections of a total of ' + str(len(vi_gp)) + ' unique objects')
print('That gives ' + str(len(vi)/len(vi_gp)) + ' inspections per object.')

desi-vi_SV0_ELG_tile67230_night20200315_11_CGS.csv
desi-vi_SV0_ELG_tile67230_night20200315_11_VRK.csv
desi-vi_SV0_ELG_tile67230_night20200315_12_DRP.csv
desi-vi_SV0_ELG_tile67230_night20200315_12_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_12_VRK.csv
desi-vi_SV0_ELG_tile67230_night20200315_13_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_13_CGS.csv
desi-vi_SV0_ELG_tile67230_night20200315_13_DRP.csv
desi-vi_SV0_ELG_tile67230_night20200315_14_DRP.csv
desi-vi_SV0_ELG_tile67230_night20200315_14_BLH.csv
desi-vi_SV0_ELG_tile67230_night20200315_14_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_15_TWL.csv
desi-vi_SV0_ELG_tile67230_night20200315_15_DRP.csv
desi-vi_SV0_ELG_tile67230_night20200315_15_BJW.csv
There are 750 visual inspections of a total of 250 unique objects
That gives 3.0 inspections per object.


In [8]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment'],
      dtype='object')

### Merge with zbest files
Add: fiberID, delta_chi2, flux information,.. anything else?


In [9]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

tspec_df = tspec['TARGETID','DELTACHI2' ].to_pandas()
tf_df = tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_pandas()

tf_df = tf_df.rename(columns={"TARGETID": "TargetID"})
tspec_df = tspec_df.rename(columns={"TARGETID": "TargetID"})

vi = vi.merge(tf_df, how='left', on='TargetID')
vi = vi.merge(tspec_df, how='left', on='TargetID')

In [10]:
print(vi.keys())
print(len(vi))

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2'],
      dtype='object')
750


### Adding a bunch of useful columns

In [11]:
#make new column with best redshift estimate for each VI - take VI redshift if available, else take Redrock redshift. 
#I am always assuming that the VI redshift, if provided, trumps over the Redrock redshift. 
vi['best redshift'] = vi['VI z']
vi.loc[vi['best redshift']=='--', 'best redshift'] = vi.loc[vi['best redshift']=='--', 'Redrock z']
vi.loc[vi['best redshift']=='Target ', 'best redshift'] = 0 #catch someone who wrote in VI z field
vi['best redshift'] = vi['best redshift'].astype(float)

In [12]:
#make new column with best spectype estimate for each VI - take VI spectype if available, else take Redrock spectype 
#I am always assuming that the VI redshift, if provided, trumps over the Redrock redshift. 
vi['best spectype'] = vi['VI spectype']
vi.loc[vi['best spectype']=='--', 'best spectype'] = vi.loc[vi['best spectype']=='--', 'Redrock spectype']

In [13]:
#add new columns, holding the mean of the flags and the maximum difference in flag classification
vi['vi_combined_flag'] = vi.groupby('TargetID')['VI class'].transform('mean')
vi['vi_diff'] = vi.groupby('TargetID')['VI class'].transform(lambda x: ( x.max()-x.min()) )

In [14]:
#add new column, difference in 'best redshift'
vi['dz'] = vi.groupby('TargetID')['best redshift'].transform(lambda x: ( (x.max() - x.min()) / (1+x.min()) ))
#add new column with the mean of best redshift if they're within 0.0033*(1+z)
vi.loc[vi.dz < 0.0033, 'vi_combined_z'] = vi.loc[vi.dz < 0.0033].groupby('TargetID')['best redshift'].transform('mean')
#keep as redrock z is difference is greater. These will be manually resolved if at least one of the inspectors flags a z as flag >2
vi.loc[vi.dz >= 0.0033, 'vi_combined_z'] = vi.loc[vi.dz >= 0.0033, 'Redrock z']

In [15]:
#add new column, with all comments concatenated
vi['all VI comments'] = vi.groupby('TargetID')['VI comment'].transform(lambda x: '|'.join(x))

In [16]:
#add new column, with the number of VI inspections for each object
vi['N_VI'] = vi.groupby('TargetID')['TargetID'].transform('count')

In [17]:
#add new column to hold comments from merger if needed
vi['merger comment'] = 'none'

In [18]:
#check all the new columns (keys) have been added correctly
display(vi[vi.dz > 0.0033])
print(vi.keys())

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
13,35185754028118319,-1,0.33.0.dev3890,0,GALAXY,0.5312,BJW,1,--,--,...,1.316707,0.5312,GALAXY,1.000000,0,0.018482,0.5312,absorption line fit is low S/N - poss continu...,3,none
46,35185754036502670,-1,0.33.0.dev3890,0,GALAXY,0.5055,BJW,0,R,--,...,20.336725,0.5055,GALAXY,1.000000,2,0.354168,0.5055,inverted template|--| RR fit not convincing +...,3,none
63,35185754028118319,-1,0.33.0.dev3890,0,GALAXY,0.5312,CGS,1,--,--,...,1.316707,0.5312,GALAXY,1.000000,0,0.018482,0.5312,absorption line fit is low S/N - poss continu...,3,none
96,35185754036502670,-1,0.33.0.dev3890,0,GALAXY,0.5055,CGS,1,--,--,...,20.336725,0.5055,GALAXY,1.000000,2,0.354168,0.5055,inverted template|--| RR fit not convincing +...,3,none
113,35185754028118319,-1,0.33.0.dev3890,0,GALAXY,0.5312,vrk,1,R,0.5595,...,1.316707,0.5595,GALAXY,1.000000,0,0.018482,0.5312,absorption line fit is low S/N - poss continu...,3,none
146,35185754036502670,-1,0.33.0.dev3890,0,GALAXY,0.5055,vrk,2,RS,1.0387,...,20.336725,1.0387,GALAXY,1.000000,2,0.354168,0.5055,inverted template|--| RR fit not convincing +...,3,none
169,35185754040700555,-1,0.33.0.dev3890,0,GALAXY,0.6656,DRP,1,--,--,...,12.283126,0.6656,GALAXY,0.666667,1,0.334534,0.6656,--| Some continuum| dubious fit (doublet and l...,3,none
170,35185754040700578,-1,0.33.0.dev3890,0,GALAXY,0.8774,DRP,2,--,0.8737,...,31.236683,0.8737,GALAXY,2.333333,1,0.346000,0.8774,--| Possible [OII] at z~1.522 next to sky line...,3,none
219,35185754040700555,-1,0.33.0.dev3890,0,GALAXY,0.6656,TWL,1,--,--,...,12.283126,0.6656,GALAXY,0.666667,1,0.334534,0.6656,--| Some continuum| dubious fit (doublet and l...,3,none
220,35185754040700578,-1,0.33.0.dev3890,0,GALAXY,0.8774,TWL,3,--,1.5220,...,31.236683,1.5220,GALAXY,2.333333,1,0.346000,0.8774,--| Possible [OII] at z~1.522 next to sky line...,3,none


Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff', 'dz',
       'vi_combined_z', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')


Get a table that holds only the objects that have been inspected more than once, and for which the individual VI classifications differ by 2 or more, or delta z / (1 + z) > 0.0033, or there is disagreement in best spectype (these are the conflicts to resolve)

In [19]:
vi_gp = vi.groupby(['TargetID'])
vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
                       & (x['VI class'].max() > 2)
                       & (len(x) >= 2)) #x is a group by TargetID

Get the target IDs of the problematic objects and display in table form for a quick summary:

In [20]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185754032309199 35185754032309750 35185754032310573 35185754032310703
 35185754032313260 35185754036503154 35185754036503778 35185754036503964
 35185754036504236 35185754036504971 35185754036505827 35185754036506000
 35185754040698310 35185754040699635 35185754040700397 35185754040700578
 35185754040701930 35185754044891804 35185754044892011 35185754044893397
 35185754044894357 35185754044895452 35185754044895763 35185754049088211
 35185754053283933 35185754053284109 35185754057477287 35185754057477995
 35185754057478517 35185754057478722 35185754061670674 35185754061671838
 35185760059523574 35185760059523701 35185760059524481 35185760063717666
 35185760063718971 35185760063720182 35185760063720299 35185760063720839
 35185760063720985 35185760067913133 35185760067913744 35185760067914129
 35185760067914335 35185760067914470 35185760067915063 35185760067915894
 35185760072106551]
Total number of conflicts to resolve:  49


In [21]:
unique_targets[1]

35185754032309750

## This is where I resolve things manually - with care!!
### I think it's better to keep it in a notebook, as typos can be backtracked rather than a single manual edit of a text file

We edit either 'VI class', or 'best redshift' to resolve conflict. At the end, we look for conflicts again and we should find none.


In [27]:
#function to display the conflict in table format and open a prospect window
def display_conflict(conflict_id, show_spectra=None):
    
    #first, remind myself of the problem:
    display(vi[vi.TargetID==unique_targets[conflict_id]][['TargetID', 
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff',
       'vi_combined_z', 'dz', 'all VI comments', 'N_VI', 'merger comment']])

    # VI interface in notebook
    if show_spectra:
        spectra, zcat= utils_specviewer.load_spectra_zcat_from_targets([unique_targets[conflict_id]], tiledir, obs_db)
        plotframes.plotspectra(spectra, zcatalog=zcat, title='Target_select', notebook=True, mask_type='CMX_TARGET',with_vi_widgets=False)

In [23]:
#first, keep a safe copy of the original dataframe
vi_safe = vi.copy()

We will inspect each conflict on a prospect window, and resolve each conflict in turn

In [31]:
#Keep track of the conflicts by conflict_id

#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
21,35185754032309199,GALAXY,1.4716,BJW,3,--,--,--,--,1.4716,GALAXY,3.0,2,1.4716,0.0,--|--|--,3,none
71,35185754032309199,GALAXY,1.4716,CGS,2,--,--,--,--,1.4716,GALAXY,3.0,2,1.4716,0.0,--|--|--,3,none
121,35185754032309199,GALAXY,1.4716,vrk,4,--,--,--,--,1.4716,GALAXY,3.0,2,1.4716,0.0,--|--|--,3,none


In [33]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'mismatch in flag, set to mean'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
21,35185754032309199,-1,0.33.0.dev3890,0,GALAXY,1.4716,BJW,3,--,--,...,408.954239,1.4716,GALAXY,3.0,2,0.0,1.4716,--|--|--,3,"mismatch in flag, set to mean"
71,35185754032309199,-1,0.33.0.dev3890,0,GALAXY,1.4716,CGS,2,--,--,...,408.954239,1.4716,GALAXY,3.0,2,0.0,1.4716,--|--|--,3,"mismatch in flag, set to mean"
121,35185754032309199,-1,0.33.0.dev3890,0,GALAXY,1.4716,vrk,4,--,--,...,408.954239,1.4716,GALAXY,3.0,2,0.0,1.4716,--|--|--,3,"mismatch in flag, set to mean"


conflict ID =  0


In [36]:
#copy this text to a new cell to display the conflict
conflict_id=1
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
27,35185754032309750,QSO,1.2374,BJW,3,--,--,--,MgII emission; Hbeta absorption/emission?,1.2374,QSO,2.0,2,1.2374,0.0,MgII emission; Hbeta absorption/emission?| QS...,3,none
77,35185754032309750,QSO,1.2374,CGS,1,--,--,--,QSO?,1.2374,QSO,2.0,2,1.2374,0.0,MgII emission; Hbeta absorption/emission?| QS...,3,none
127,35185754032309750,QSO,1.2374,vrk,2,--,--,--,QSO but CIII feature might not be secure,1.2374,QSO,2.0,2,1.2374,0.0,MgII emission; Hbeta absorption/emission?| QS...,3,none


In [37]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'broad emission line, but odd continuum. disagreement in flag, take the mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
27,35185754032309750,-1,0.33.0.dev3890,0,QSO,1.2374,BJW,2,--,--,...,48.094389,1.2374,QSO,2.0,2,0.0,1.2374,MgII emission; Hbeta absorption/emission?| QS...,3,"broad emission line, but odd continuum. disagr..."
77,35185754032309750,-1,0.33.0.dev3890,0,QSO,1.2374,CGS,2,--,--,...,48.094389,1.2374,QSO,2.0,2,0.0,1.2374,MgII emission; Hbeta absorption/emission?| QS...,3,"broad emission line, but odd continuum. disagr..."
127,35185754032309750,-1,0.33.0.dev3890,0,QSO,1.2374,vrk,2,--,--,...,48.094389,1.2374,QSO,2.0,2,0.0,1.2374,MgII emission; Hbeta absorption/emission?| QS...,3,"broad emission line, but odd continuum. disagr..."


conflict ID =  1


In [40]:
#copy this text to a new cell to display the conflict
conflict_id = 2
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
31,35185754032310573,GALAXY,1.4879,BJW,3,--,--,--,--,1.4879,GALAXY,3.0,2,1.4879,0.0,--|--|--,3,none
81,35185754032310573,GALAXY,1.4879,CGS,2,--,--,--,--,1.4879,GALAXY,3.0,2,1.4879,0.0,--|--|--,3,none
131,35185754032310573,GALAXY,1.4879,vrk,4,--,--,--,--,1.4879,GALAXY,3.0,2,1.4879,0.0,--|--|--,3,none


In [41]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. no continuum but OII secures z. take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
31,35185754032310573,-1,0.33.0.dev3890,0,GALAXY,1.4879,BJW,3,--,--,...,81.357731,1.4879,GALAXY,3.0,2,0.0,1.4879,--|--|--,3,disagreement in flag. no continuum but OII sec...
81,35185754032310573,-1,0.33.0.dev3890,0,GALAXY,1.4879,CGS,3,--,--,...,81.357731,1.4879,GALAXY,3.0,2,0.0,1.4879,--|--|--,3,disagreement in flag. no continuum but OII sec...
131,35185754032310573,-1,0.33.0.dev3890,0,GALAXY,1.4879,vrk,3,--,--,...,81.357731,1.4879,GALAXY,3.0,2,0.0,1.4879,--|--|--,3,disagreement in flag. no continuum but OII sec...


conflict ID =  2


In [44]:
#copy this text to a new cell to display the conflict
conflict_id= 3
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
32,35185754032310703,GALAXY,1.5176,BJW,3,--,--,--,--,1.5176,GALAXY,2.666667,3,1.5176,0.0,--|--|--,3,none
82,35185754032310703,GALAXY,1.5176,CGS,1,--,--,--,--,1.5176,GALAXY,2.666667,3,1.5176,0.0,--|--|--,3,none
132,35185754032310703,GALAXY,1.5176,vrk,4,--,--,--,--,1.5176,GALAXY,2.666667,3,1.5176,0.0,--|--|--,3,none


In [46]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.666667
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'mismatch in flag. OII is there and just misses a skyline but its not fully resolved. mean flag 2.7'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
32,35185754032310703,-1,0.33.0.dev3890,0,GALAXY,1.5176,BJW,2.666667,--,--,...,101.810069,1.5176,GALAXY,2.666667,3,0.0,1.5176,--|--|--,3,mismatch in flag. OII is there and just misses...
82,35185754032310703,-1,0.33.0.dev3890,0,GALAXY,1.5176,CGS,2.666667,--,--,...,101.810069,1.5176,GALAXY,2.666667,3,0.0,1.5176,--|--|--,3,mismatch in flag. OII is there and just misses...
132,35185754032310703,-1,0.33.0.dev3890,0,GALAXY,1.5176,vrk,2.666667,--,--,...,101.810069,1.5176,GALAXY,2.666667,3,0.0,1.5176,--|--|--,3,mismatch in flag. OII is there and just misses...


conflict ID =  3


In [51]:
#copy this text to a new cell to display the conflict
conflict_id=4
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
43,35185754032313260,GALAXY,1.3582,BJW,3.0,--,--,--,--,1.3582,GALAXY,3.0,2,1.3582,0.0,--|--|--,3,none
93,35185754032313260,GALAXY,1.3582,CGS,2.0,--,--,--,--,1.3582,GALAXY,3.0,2,1.3582,0.0,--|--|--,3,none
143,35185754032313260,GALAXY,1.3582,vrk,4.0,--,--,--,--,1.3582,GALAXY,3.0,2,1.3582,0.0,--|--|--,3,none


In [52]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. no continuum but OII just about resolved. set to mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
43,35185754032313260,-1,0.33.0.dev3890,0,GALAXY,1.3582,BJW,3.0,--,--,...,294.239779,1.3582,GALAXY,3.0,2,0.0,1.3582,--|--|--,3,disagreement in flag. no continuum but OII jus...
93,35185754032313260,-1,0.33.0.dev3890,0,GALAXY,1.3582,CGS,3.0,--,--,...,294.239779,1.3582,GALAXY,3.0,2,0.0,1.3582,--|--|--,3,disagreement in flag. no continuum but OII jus...
143,35185754032313260,-1,0.33.0.dev3890,0,GALAXY,1.3582,vrk,3.0,--,--,...,294.239779,1.3582,GALAXY,3.0,2,0.0,1.3582,--|--|--,3,disagreement in flag. no continuum but OII jus...


conflict ID =  4


In [55]:
#copy this text to a new cell to display the conflict
conflict_id=5
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
1,35185754036503154,GALAXY,1.4978,BJW,2.0,--,--,--,[OII] in sky lines,1.4978,GALAXY,2.333333,3,1.4978,0.0,[OII] in sky lines|--|--,3,none
51,35185754036503154,GALAXY,1.4978,CGS,1.0,--,--,--,--,1.4978,GALAXY,2.333333,3,1.4978,0.0,[OII] in sky lines|--|--,3,none
101,35185754036503154,GALAXY,1.4978,vrk,4.0,--,--,--,--,1.4978,GALAXY,2.333333,3,1.4978,0.0,[OII] in sky lines|--|--,3,none


In [56]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. OII doublet on skylines, not secure. set to mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
1,35185754036503154,-1,0.33.0.dev3890,0,GALAXY,1.4978,BJW,2.333333,--,--,...,49.732391,1.4978,GALAXY,2.333333,3,0.0,1.4978,[OII] in sky lines|--|--,3,"disagreement in flag. OII doublet on skylines,..."
51,35185754036503154,-1,0.33.0.dev3890,0,GALAXY,1.4978,CGS,2.333333,--,--,...,49.732391,1.4978,GALAXY,2.333333,3,0.0,1.4978,[OII] in sky lines|--|--,3,"disagreement in flag. OII doublet on skylines,..."
101,35185754036503154,-1,0.33.0.dev3890,0,GALAXY,1.4978,vrk,2.333333,--,--,...,49.732391,1.4978,GALAXY,2.333333,3,0.0,1.4978,[OII] in sky lines|--|--,3,"disagreement in flag. OII doublet on skylines,..."


conflict ID =  5


In [59]:
#copy this text to a new cell to display the conflict
conflict_id=6
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
177,35185754036503778,GALAXY,1.2195,DRP,2.0,--,--,--,--,1.2195,GALAXY,3.333333,2,1.2195,0.0,--| Triple-peak [OII] emission|--,3,none
227,35185754036503778,GALAXY,1.2195,TWL,4.0,--,--,--,Triple-peak [OII] emission,1.2195,GALAXY,3.333333,2,1.2195,0.0,--| Triple-peak [OII] emission|--,3,none
277,35185754036503778,GALAXY,1.2195,vrk,4.0,--,--,--,--,1.2195,GALAXY,3.333333,2,1.2195,0.0,--| Triple-peak [OII] emission|--,3,none


In [60]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag as OII is not fully resolved. set to mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
177,35185754036503778,-1,0.33.0.dev3890,0,GALAXY,1.2195,DRP,3.333333,--,--,...,204.38953,1.2195,GALAXY,3.333333,2,0.0,1.2195,--| Triple-peak [OII] emission|--,3,disagreement in flag as OII is not fully resol...
227,35185754036503778,-1,0.33.0.dev3890,0,GALAXY,1.2195,TWL,3.333333,--,--,...,204.38953,1.2195,GALAXY,3.333333,2,0.0,1.2195,--| Triple-peak [OII] emission|--,3,disagreement in flag as OII is not fully resol...
277,35185754036503778,-1,0.33.0.dev3890,0,GALAXY,1.2195,vrk,3.333333,--,--,...,204.38953,1.2195,GALAXY,3.333333,2,0.0,1.2195,--| Triple-peak [OII] emission|--,3,disagreement in flag as OII is not fully resol...


conflict ID =  6


In [63]:
#copy this text to a new cell to display the conflict
conflict_id=7
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
179,35185754036503964,GALAXY,1.5405,DRP,1.0,--,--,--,--,1.5405,GALAXY,2.0,2,1.5405,0.0,--| Possible triple peak [OII] between sky lin...,3,none
229,35185754036503964,GALAXY,1.5405,TWL,3.0,--,--,--,Possible triple peak [OII] between sky lines,1.5405,GALAXY,2.0,2,1.5405,0.0,--| Possible triple peak [OII] between sky lin...,3,none
279,35185754036503964,GALAXY,1.5405,vrk,2.0,--,--,--,dubious doublet in skyline noise + no signal ...,1.5405,GALAXY,2.0,2,1.5405,0.0,--| Possible triple peak [OII] between sky lin...,3,none


In [64]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement due to possible doublet in skylines. set to mean flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
179,35185754036503964,-1,0.33.0.dev3890,0,GALAXY,1.5405,DRP,2.0,--,--,...,42.366031,1.5405,GALAXY,2.0,2,0.0,1.5405,--| Possible triple peak [OII] between sky lin...,3,flag disagreement due to possible doublet in s...
229,35185754036503964,-1,0.33.0.dev3890,0,GALAXY,1.5405,TWL,2.0,--,--,...,42.366031,1.5405,GALAXY,2.0,2,0.0,1.5405,--| Possible triple peak [OII] between sky lin...,3,flag disagreement due to possible doublet in s...
279,35185754036503964,-1,0.33.0.dev3890,0,GALAXY,1.5405,vrk,2.0,--,--,...,42.366031,1.5405,GALAXY,2.0,2,0.0,1.5405,--| Possible triple peak [OII] between sky lin...,3,flag disagreement due to possible doublet in s...


conflict ID =  7


In [67]:
#copy this text to a new cell to display the conflict
conflict_id= 8
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
180,35185754036504236,GALAXY,1.1705,DRP,2.0,--,--,--,--,1.1705,GALAXY,3.333333,2,1.1705,0.0,--|--|--,3,none
230,35185754036504236,GALAXY,1.1705,TWL,4.0,--,--,--,--,1.1705,GALAXY,3.333333,2,1.1705,0.0,--|--|--,3,none
280,35185754036504236,GALAXY,1.1705,vrk,4.0,--,--,--,--,1.1705,GALAXY,3.333333,2,1.1705,0.0,--|--|--,3,none


In [68]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, but doublet secures z'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
180,35185754036504236,-1,0.33.0.dev3890,0,GALAXY,1.1705,DRP,4.0,--,--,...,159.244242,1.1705,GALAXY,3.333333,2,0.0,1.1705,--|--|--,3,"disagreement in flag, but doublet secures z"
230,35185754036504236,-1,0.33.0.dev3890,0,GALAXY,1.1705,TWL,4.0,--,--,...,159.244242,1.1705,GALAXY,3.333333,2,0.0,1.1705,--|--|--,3,"disagreement in flag, but doublet secures z"
280,35185754036504236,-1,0.33.0.dev3890,0,GALAXY,1.1705,vrk,4.0,--,--,...,159.244242,1.1705,GALAXY,3.333333,2,0.0,1.1705,--|--|--,3,"disagreement in flag, but doublet secures z"


conflict ID =  8


In [71]:
#copy this text to a new cell to display the conflict
conflict_id=9
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
184,35185754036504971,QSO,2.8897,DRP,3.0,--,--,--,--,2.8897,QSO,3.0,2,2.8897,0.0,--|Not sure the model is a good fit| Not an EL...,3,none
234,35185754036504971,QSO,2.8897,TWL,2.0,--,--,QSO,Not sure the model is a good fit,2.8897,QSO,3.0,2,2.8897,0.0,--|Not sure the model is a good fit| Not an EL...,3,none
284,35185754036504971,QSO,2.8897,vrk,4.0,--,--,--,Not an ELG but QSO,2.8897,QSO,3.0,2,2.8897,0.0,--|Not sure the model is a good fit| Not an EL...,3,none


In [72]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. no secure features.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
184,35185754036504971,-1,0.33.0.dev3890,0,QSO,2.8897,DRP,2.0,--,--,...,11.481968,2.8897,QSO,3.0,2,0.0,2.8897,--|Not sure the model is a good fit| Not an EL...,3,disagreement in flag. no secure features.
234,35185754036504971,-1,0.33.0.dev3890,0,QSO,2.8897,TWL,2.0,--,--,...,11.481968,2.8897,QSO,3.0,2,0.0,2.8897,--|Not sure the model is a good fit| Not an EL...,3,disagreement in flag. no secure features.
284,35185754036504971,-1,0.33.0.dev3890,0,QSO,2.8897,vrk,2.0,--,--,...,11.481968,2.8897,QSO,3.0,2,0.0,2.8897,--|Not sure the model is a good fit| Not an EL...,3,disagreement in flag. no secure features.


conflict ID =  9


In [75]:
#copy this text to a new cell to display the conflict
conflict_id=10
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
188,35185754036505827,GALAXY,0.1982,DRP,3.0,--,0.1986,--,--,0.1986,GALAXY,2.0,2,0.198333,0.000334,--| Low S/N; hard to tell| fit based only on t...,3,none
238,35185754036505827,GALAXY,0.1982,TWL,1.0,--,--,--,Low S/N; hard to tell,0.1982,GALAXY,2.0,2,0.198333,0.000334,--| Low S/N; hard to tell| fit based only on t...,3,none
288,35185754036505827,GALAXY,0.1982,vrk,2.0,--,--,--,fit based only on two weak OIII lines + doubl...,0.1982,GALAXY,2.0,2,0.198333,0.000334,--| Low S/N; hard to tell| fit based only on t...,3,none


In [76]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.1615
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. tentative VI on OII, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
188,35185754036505827,-1,0.33.0.dev3890,0,GALAXY,0.1982,DRP,1.0,--,0.1986,...,7.216114,0.1615,GALAXY,2.0,2,0.000334,0.198333,--| Low S/N; hard to tell| fit based only on t...,3,"disagreement in flag. tentative VI on OII, fla..."
238,35185754036505827,-1,0.33.0.dev3890,0,GALAXY,0.1982,TWL,1.0,--,--,...,7.216114,0.1615,GALAXY,2.0,2,0.000334,0.198333,--| Low S/N; hard to tell| fit based only on t...,3,"disagreement in flag. tentative VI on OII, fla..."
288,35185754036505827,-1,0.33.0.dev3890,0,GALAXY,0.1982,vrk,1.0,--,--,...,7.216114,0.1615,GALAXY,2.0,2,0.000334,0.198333,--| Low S/N; hard to tell| fit based only on t...,3,"disagreement in flag. tentative VI on OII, fla..."


conflict ID =  10


In [79]:
#copy this text to a new cell to display the conflict
conflict_id=11
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
190,35185754036506000,GALAXY,0.8734,DRP,2.0,--,--,--,--,0.8734,GALAXY,3.333333,2,0.8734,0.0,--| Broad [OII] emission|--,3,none
240,35185754036506000,GALAXY,0.8734,TWL,4.0,--,--,--,Broad [OII] emission,0.8734,GALAXY,3.333333,2,0.8734,0.0,--| Broad [OII] emission|--,3,none
290,35185754036506000,GALAXY,0.8734,vrk,4.0,--,--,--,--,0.8734,GALAXY,3.333333,2,0.8734,0.0,--| Broad [OII] emission|--,3,none


In [80]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag due to borderline resolved OII. set to mean 3.3'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
190,35185754036506000,-1,0.33.0.dev3890,0,GALAXY,0.8734,DRP,3.333333,--,--,...,259.121594,0.8734,GALAXY,3.333333,2,0.0,0.8734,--| Broad [OII] emission|--,3,disagreement in flag due to borderline resolve...
240,35185754036506000,-1,0.33.0.dev3890,0,GALAXY,0.8734,TWL,3.333333,--,--,...,259.121594,0.8734,GALAXY,3.333333,2,0.0,0.8734,--| Broad [OII] emission|--,3,disagreement in flag due to borderline resolve...
290,35185754036506000,-1,0.33.0.dev3890,0,GALAXY,0.8734,vrk,3.333333,--,--,...,259.121594,0.8734,GALAXY,3.333333,2,0.0,0.8734,--| Broad [OII] emission|--,3,disagreement in flag due to borderline resolve...


conflict ID =  11


In [83]:
#copy this text to a new cell to display the conflict
conflict_id=12
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
156,35185754040698310,GALAXY,1.1667,DRP,2.0,--,--,--,--,1.1667,GALAXY,3.333333,2,1.1667,0.0,--|--|--,3,none
206,35185754040698310,GALAXY,1.1667,TWL,4.0,--,--,--,--,1.1667,GALAXY,3.333333,2,1.1667,0.0,--|--|--,3,none
256,35185754040698310,GALAXY,1.1667,vrk,4.0,--,--,--,--,1.1667,GALAXY,3.333333,2,1.1667,0.0,--|--|--,3,none


In [84]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, but doublet secures z'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
156,35185754040698310,-1,0.33.0.dev3890,0,GALAXY,1.1667,DRP,4.0,--,--,...,94.727684,1.1667,GALAXY,3.333333,2,0.0,1.1667,--|--|--,3,"disagreement in flag, but doublet secures z"
206,35185754040698310,-1,0.33.0.dev3890,0,GALAXY,1.1667,TWL,4.0,--,--,...,94.727684,1.1667,GALAXY,3.333333,2,0.0,1.1667,--|--|--,3,"disagreement in flag, but doublet secures z"
256,35185754040698310,-1,0.33.0.dev3890,0,GALAXY,1.1667,vrk,4.0,--,--,...,94.727684,1.1667,GALAXY,3.333333,2,0.0,1.1667,--|--|--,3,"disagreement in flag, but doublet secures z"


conflict ID =  12


In [87]:
#copy this text to a new cell to display the conflict
conflict_id=13
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
163,35185754040699635,GALAXY,0.2795,DRP,3.0,--,--,--,--,0.2795,GALAXY,1.333333,3,0.2795,0.0,--| Some continuum| dubious fit (doublet and o...,3,none
213,35185754040699635,GALAXY,0.2795,TWL,1.0,--,--,--,Some continuum,0.2795,GALAXY,1.333333,3,0.2795,0.0,--| Some continuum| dubious fit (doublet and o...,3,none
263,35185754040699635,GALAXY,0.2795,vrk,0.0,R,--,--,dubious fit (doublet and one OIII line in abs...,0.2795,GALAXY,1.333333,3,0.2795,0.0,--| Some continuum| dubious fit (doublet and o...,3,none


In [89]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. no secure features, flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
163,35185754040699635,-1,0.33.0.dev3890,0,GALAXY,0.2795,DRP,2.0,--,--,...,2.172872,0.2795,GALAXY,1.333333,3,0.0,0.2795,--| Some continuum| dubious fit (doublet and o...,3,"disagreement in flag. no secure features, flag 2."
213,35185754040699635,-1,0.33.0.dev3890,0,GALAXY,0.2795,TWL,2.0,--,--,...,2.172872,0.2795,GALAXY,1.333333,3,0.0,0.2795,--| Some continuum| dubious fit (doublet and o...,3,"disagreement in flag. no secure features, flag 2."
263,35185754040699635,-1,0.33.0.dev3890,0,GALAXY,0.2795,vrk,2.0,R,--,...,2.172872,0.2795,GALAXY,1.333333,3,0.0,0.2795,--| Some continuum| dubious fit (doublet and o...,3,"disagreement in flag. no secure features, flag 2."


conflict ID =  13


In [92]:
#copy this text to a new cell to display the conflict
conflict_id=14
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
167,35185754040700397,GALAXY,1.3009,DRP,2.0,--,--,--,--,1.3009,GALAXY,3.333333,2,1.3009,0.0,--|--|--,3,none
217,35185754040700397,GALAXY,1.3009,TWL,4.0,--,--,--,--,1.3009,GALAXY,3.333333,2,1.3009,0.0,--|--|--,3,none
267,35185754040700397,GALAXY,1.3009,vrk,4.0,--,--,--,--,1.3009,GALAXY,3.333333,2,1.3009,0.0,--|--|--,3,none


In [93]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, but doublet secures z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
167,35185754040700397,-1,0.33.0.dev3890,0,GALAXY,1.3009,DRP,4.0,--,--,...,295.672356,1.3009,GALAXY,3.333333,2,0.0,1.3009,--|--|--,3,"disagreement in flag, but doublet secures z."
217,35185754040700397,-1,0.33.0.dev3890,0,GALAXY,1.3009,TWL,4.0,--,--,...,295.672356,1.3009,GALAXY,3.333333,2,0.0,1.3009,--|--|--,3,"disagreement in flag, but doublet secures z."
267,35185754040700397,-1,0.33.0.dev3890,0,GALAXY,1.3009,vrk,4.0,--,--,...,295.672356,1.3009,GALAXY,3.333333,2,0.0,1.3009,--|--|--,3,"disagreement in flag, but doublet secures z."


conflict ID =  14


In [96]:
#copy this text to a new cell to display the conflict
conflict_id=15
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
170,35185754040700578,GALAXY,0.8774,DRP,2.0,--,0.8737,--,--,0.8737,GALAXY,2.333333,1,0.8774,0.346,--| Possible [OII] at z~1.522 next to sky line...,3,none
220,35185754040700578,GALAXY,0.8774,TWL,3.0,--,1.5220,--,Possible [OII] at z~1.522 next to sky lines,1.5220,GALAXY,2.333333,1,0.8774,0.346,--| Possible [OII] at z~1.522 next to sky line...,3,none
270,35185754040700578,GALAXY,0.8774,vrk,2.0,R,1.5220,--,dubious fit driven by one OIII line in skylin...,1.5220,GALAXY,2.333333,1,0.8774,0.346,--| Possible [OII] at z~1.522 next to sky line...,3,none


In [97]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.5220
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. redrock z is wrong. VI z more likely but doublet compromised by skyline. give mean flag of two inspectors at 2.5'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
170,35185754040700578,-1,0.33.0.dev3890,0,GALAXY,0.8774,DRP,2.5,--,0.8737,...,31.236683,1.522,GALAXY,2.333333,1,0.346,0.8774,--| Possible [OII] at z~1.522 next to sky line...,3,disagreement in z and flag. redrock z is wrong...
220,35185754040700578,-1,0.33.0.dev3890,0,GALAXY,0.8774,TWL,2.5,--,1.5220,...,31.236683,1.522,GALAXY,2.333333,1,0.346,0.8774,--| Possible [OII] at z~1.522 next to sky line...,3,disagreement in z and flag. redrock z is wrong...
270,35185754040700578,-1,0.33.0.dev3890,0,GALAXY,0.8774,vrk,2.5,R,1.5220,...,31.236683,1.522,GALAXY,2.333333,1,0.346,0.8774,--| Possible [OII] at z~1.522 next to sky line...,3,disagreement in z and flag. redrock z is wrong...


conflict ID =  15


In [100]:
#copy this text to a new cell to display the conflict
conflict_id=16
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
337,35185754040701930,GALAXY,1.4666,TWL,4.0,--,--,--,--,1.4666,GALAXY,3.0,2,1.4666,0.0,--|--|--,3,none
387,35185754040701930,GALAXY,1.4666,CGS,3.0,--,--,--,--,1.4666,GALAXY,3.0,2,1.4666,0.0,--|--|--,3,none
437,35185754040701930,GALAXY,1.4666,DRP,2.0,--,--,--,--,1.4666,GALAXY,3.0,2,1.4666,0.0,--|--|--,3,none


In [101]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, but OII doublet secures z'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
337,35185754040701930,-1,0.33.0.dev3890,0,GALAXY,1.4666,TWL,4.0,--,--,...,42.852848,1.4666,GALAXY,3.0,2,0.0,1.4666,--|--|--,3,"disagreement in flag, but OII doublet secures z"
387,35185754040701930,-1,0.33.0.dev3890,0,GALAXY,1.4666,CGS,4.0,--,--,...,42.852848,1.4666,GALAXY,3.0,2,0.0,1.4666,--|--|--,3,"disagreement in flag, but OII doublet secures z"
437,35185754040701930,-1,0.33.0.dev3890,0,GALAXY,1.4666,DRP,4.0,--,--,...,42.852848,1.4666,GALAXY,3.0,2,0.0,1.4666,--|--|--,3,"disagreement in flag, but OII doublet secures z"


conflict ID =  16


In [104]:
#copy this text to a new cell to display the conflict
conflict_id=17
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
305,35185754044891804,GALAXY,1.0499,TWL,4.0,--,--,--,--,1.0499,GALAXY,3.0,2,1.0499,0.0,--|--|--,3,none
355,35185754044891804,GALAXY,1.0499,CGS,3.0,--,--,--,--,1.0499,GALAXY,3.0,2,1.0499,0.0,--|--|--,3,none
405,35185754044891804,GALAXY,1.0499,DRP,2.0,--,--,--,--,1.0499,GALAXY,3.0,2,1.0499,0.0,--|--|--,3,none


In [105]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, but resolved doublet secures z'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
305,35185754044891804,-1,0.33.0.dev3890,0,GALAXY,1.0499,TWL,4.0,--,--,...,67.141016,1.0499,GALAXY,3.0,2,0.0,1.0499,--|--|--,3,"disagreement in flag, but resolved doublet sec..."
355,35185754044891804,-1,0.33.0.dev3890,0,GALAXY,1.0499,CGS,4.0,--,--,...,67.141016,1.0499,GALAXY,3.0,2,0.0,1.0499,--|--|--,3,"disagreement in flag, but resolved doublet sec..."
405,35185754044891804,-1,0.33.0.dev3890,0,GALAXY,1.0499,DRP,4.0,--,--,...,67.141016,1.0499,GALAXY,3.0,2,0.0,1.0499,--|--|--,3,"disagreement in flag, but resolved doublet sec..."


conflict ID =  17


In [108]:
#copy this text to a new cell to display the conflict
conflict_id=18
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
307,35185754044892011,GALAXY,1.4243,TWL,4.0,--,--,--,--,1.4243,GALAXY,3.0,2,1.4243,0.0,--|--|--,3,none
357,35185754044892011,GALAXY,1.4243,CGS,3.0,--,--,--,--,1.4243,GALAXY,3.0,2,1.4243,0.0,--|--|--,3,none
407,35185754044892011,GALAXY,1.4243,DRP,2.0,--,--,--,--,1.4243,GALAXY,3.0,2,1.4243,0.0,--|--|--,3,none


In [109]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, but resolved doublet secures z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
307,35185754044892011,-1,0.33.0.dev3890,0,GALAXY,1.4243,TWL,3.0,--,--,...,55.153214,1.4243,GALAXY,3.0,2,0.0,1.4243,--|--|--,3,"disagreement in flag, but resolved doublet sec..."
357,35185754044892011,-1,0.33.0.dev3890,0,GALAXY,1.4243,CGS,3.0,--,--,...,55.153214,1.4243,GALAXY,3.0,2,0.0,1.4243,--|--|--,3,"disagreement in flag, but resolved doublet sec..."
407,35185754044892011,-1,0.33.0.dev3890,0,GALAXY,1.4243,DRP,3.0,--,--,...,55.153214,1.4243,GALAXY,3.0,2,0.0,1.4243,--|--|--,3,"disagreement in flag, but resolved doublet sec..."


conflict ID =  18


In [112]:
#copy this text to a new cell to display the conflict
conflict_id=19
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
315,35185754044893397,GALAXY,1.0882,TWL,4.0,--,--,--,Broad [OII] emission,1.0882,GALAXY,3.0,2,1.0882,0.0,Broad [OII] emission|--|--,3,none
365,35185754044893397,GALAXY,1.0882,CGS,3.0,--,--,--,--,1.0882,GALAXY,3.0,2,1.0882,0.0,Broad [OII] emission|--|--,3,none
415,35185754044893397,GALAXY,1.0882,DRP,2.0,--,--,--,--,1.0882,GALAXY,3.0,2,1.0882,0.0,Broad [OII] emission|--|--,3,none


In [113]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. Broadened but not resolved OII. mean flag 3.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
315,35185754044893397,-1,0.33.0.dev3890,0,GALAXY,1.0882,TWL,3.0,--,--,...,130.930602,1.0882,GALAXY,3.0,2,0.0,1.0882,Broad [OII] emission|--|--,3,flag disagreement. Broadened but not resolved ...
365,35185754044893397,-1,0.33.0.dev3890,0,GALAXY,1.0882,CGS,3.0,--,--,...,130.930602,1.0882,GALAXY,3.0,2,0.0,1.0882,Broad [OII] emission|--|--,3,flag disagreement. Broadened but not resolved ...
415,35185754044893397,-1,0.33.0.dev3890,0,GALAXY,1.0882,DRP,3.0,--,--,...,130.930602,1.0882,GALAXY,3.0,2,0.0,1.0882,Broad [OII] emission|--|--,3,flag disagreement. Broadened but not resolved ...


conflict ID =  19


In [116]:
#copy this text to a new cell to display the conflict
conflict_id=20
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
324,35185754044894357,GALAXY,1.1555,TWL,4.0,--,--,--,--,1.1555,GALAXY,3.333333,2,1.1555,0.0,--|--|--,3,none
374,35185754044894357,GALAXY,1.1555,CGS,4.0,--,--,--,--,1.1555,GALAXY,3.333333,2,1.1555,0.0,--|--|--,3,none
424,35185754044894357,GALAXY,1.1555,DRP,2.0,--,--,--,--,1.1555,GALAXY,3.333333,2,1.1555,0.0,--|--|--,3,none


In [117]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement but doublet secures z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
324,35185754044894357,-1,0.33.0.dev3890,0,GALAXY,1.1555,TWL,4.0,--,--,...,598.815497,1.1555,GALAXY,3.333333,2,0.0,1.1555,--|--|--,3,flag disagreement but doublet secures z.
374,35185754044894357,-1,0.33.0.dev3890,0,GALAXY,1.1555,CGS,4.0,--,--,...,598.815497,1.1555,GALAXY,3.333333,2,0.0,1.1555,--|--|--,3,flag disagreement but doublet secures z.
424,35185754044894357,-1,0.33.0.dev3890,0,GALAXY,1.1555,DRP,4.0,--,--,...,598.815497,1.1555,GALAXY,3.333333,2,0.0,1.1555,--|--|--,3,flag disagreement but doublet secures z.


conflict ID =  20


In [120]:
#copy this text to a new cell to display the conflict
conflict_id=21
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
330,35185754044895452,GALAXY,1.0359,TWL,4.0,--,--,--,--,1.0359,GALAXY,3.333333,2,1.0359,0.0,--|--|--,3,none
380,35185754044895452,GALAXY,1.0359,CGS,4.0,--,--,--,--,1.0359,GALAXY,3.333333,2,1.0359,0.0,--|--|--,3,none
430,35185754044895452,GALAXY,1.0359,DRP,2.0,--,--,--,--,1.0359,GALAXY,3.333333,2,1.0359,0.0,--|--|--,3,none


In [121]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z, but doublet secures z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
330,35185754044895452,-1,0.33.0.dev3890,0,GALAXY,1.0359,TWL,4.0,--,--,...,68.232301,1.0359,GALAXY,3.333333,2,0.0,1.0359,--|--|--,3,"disagreement in z, but doublet secures z."
380,35185754044895452,-1,0.33.0.dev3890,0,GALAXY,1.0359,CGS,4.0,--,--,...,68.232301,1.0359,GALAXY,3.333333,2,0.0,1.0359,--|--|--,3,"disagreement in z, but doublet secures z."
430,35185754044895452,-1,0.33.0.dev3890,0,GALAXY,1.0359,DRP,4.0,--,--,...,68.232301,1.0359,GALAXY,3.333333,2,0.0,1.0359,--|--|--,3,"disagreement in z, but doublet secures z."


conflict ID =  21


In [125]:
#copy this text to a new cell to display the conflict
conflict_id=22
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
332,35185754044895763,GALAXY,0.081,TWL,1.0,S,--,--,Model fits sky residuals,0.081,GALAXY,2.0,3,0.081,0.0,Model fits sky residuals|--|--,3,none
382,35185754044895763,GALAXY,0.081,CGS,1.0,--,--,--,--,0.081,GALAXY,2.0,3,0.081,0.0,Model fits sky residuals|--|--,3,none
432,35185754044895763,GALAXY,0.081,DRP,4.0,--,--,--,--,0.081,GALAXY,2.0,3,0.081,0.0,Model fits sky residuals|--|--,3,none


In [126]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag but no secure features. flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
332,35185754044895763,-1,0.33.0.dev3890,0,GALAXY,0.081,TWL,1.0,S,--,...,39.657687,0.081,GALAXY,2.0,3,0.0,0.081,Model fits sky residuals|--|--,3,disagreement in flag but no secure features. f...
382,35185754044895763,-1,0.33.0.dev3890,0,GALAXY,0.081,CGS,1.0,--,--,...,39.657687,0.081,GALAXY,2.0,3,0.0,0.081,Model fits sky residuals|--|--,3,disagreement in flag but no secure features. f...
432,35185754044895763,-1,0.33.0.dev3890,0,GALAXY,0.081,DRP,1.0,--,--,...,39.657687,0.081,GALAXY,2.0,3,0.0,0.081,Model fits sky residuals|--|--,3,disagreement in flag but no secure features. f...


conflict ID =  22


In [129]:
#copy this text to a new cell to display the conflict
conflict_id=23
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
345,35185754049088211,QSO,1.79,TWL,1.0,--,--,--,Low S/N,1.79,QSO,1.666667,2,1.79,0.071685,Low S/N| QSO suspect bad reshift|--,3,none
395,35185754049088211,QSO,1.79,CGS,3.0,R,1.9900,--,QSO suspect bad reshift,1.99,QSO,1.666667,2,1.79,0.071685,Low S/N| QSO suspect bad reshift|--,3,none
445,35185754049088211,QSO,1.79,DRP,1.0,--,--,--,--,1.79,QSO,1.666667,2,1.79,0.071685,Low S/N| QSO suspect bad reshift|--,3,none


In [130]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.9900
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. issues in calibration, no secure features. take VI z with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
345,35185754049088211,-1,0.33.0.dev3890,0,QSO,1.79,TWL,1.0,--,--,...,3.693236,1.99,QSO,1.666667,2,0.071685,1.79,Low S/N| QSO suspect bad reshift|--,3,disagreement in z and flag. issues in calibrat...
395,35185754049088211,-1,0.33.0.dev3890,0,QSO,1.79,CGS,1.0,R,1.9900,...,3.693236,1.99,QSO,1.666667,2,0.071685,1.79,Low S/N| QSO suspect bad reshift|--,3,disagreement in z and flag. issues in calibrat...
445,35185754049088211,-1,0.33.0.dev3890,0,QSO,1.79,DRP,1.0,--,--,...,3.693236,1.99,QSO,1.666667,2,0.071685,1.79,Low S/N| QSO suspect bad reshift|--,3,disagreement in z and flag. issues in calibrat...


conflict ID =  23


In [133]:
#copy this text to a new cell to display the conflict
conflict_id=24
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
464,35185754053283933,GALAXY,0.3208,DRP,2.0,--,--,--,--,0.3208,GALAXY,1.666667,3,0.3208,0.552847,--|--| Calibration problem,3,none
514,35185754053283933,GALAXY,0.3208,BLH,3.0,RS,1.0510,--,--,1.0510,GALAXY,1.666667,3,0.3208,0.552847,--|--| Calibration problem,3,none
564,35185754053283933,GALAXY,0.3208,TWL,0.0,S,--,--,Calibration problem,0.3208,GALAXY,1.666667,3,0.3208,0.552847,--|--| Calibration problem,3,none


In [134]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0510
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'issues in calibration. tentative fit to doublet.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
464,35185754053283933,-1,0.33.0.dev3890,0,GALAXY,0.3208,DRP,2.0,--,--,...,20.38032,1.051,GALAXY,1.666667,3,0.552847,0.3208,--|--| Calibration problem,3,issues in calibration. tentative fit to doublet.
514,35185754053283933,-1,0.33.0.dev3890,0,GALAXY,0.3208,BLH,2.0,RS,1.0510,...,20.38032,1.051,GALAXY,1.666667,3,0.552847,0.3208,--|--| Calibration problem,3,issues in calibration. tentative fit to doublet.
564,35185754053283933,-1,0.33.0.dev3890,0,GALAXY,0.3208,TWL,2.0,S,--,...,20.38032,1.051,GALAXY,1.666667,3,0.552847,0.3208,--|--| Calibration problem,3,issues in calibration. tentative fit to doublet.


conflict ID =  24


In [137]:
#copy this text to a new cell to display the conflict
conflict_id=25
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
466,35185754053284109,GALAXY,1.1543,DRP,2.0,--,--,--,--,1.1543,GALAXY,3.0,2,1.1543,0.0,--|--|--,3,none
516,35185754053284109,GALAXY,1.1543,BLH,3.0,--,--,--,--,1.1543,GALAXY,3.0,2,1.1543,0.0,--|--|--,3,none
566,35185754053284109,GALAXY,1.1543,TWL,4.0,--,--,--,--,1.1543,GALAXY,3.0,2,1.1543,0.0,--|--|--,3,none


In [138]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. doublet partially on sky line. set to mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
466,35185754053284109,-1,0.33.0.dev3890,0,GALAXY,1.1543,DRP,3.0,--,--,...,85.516308,1.1543,GALAXY,3.0,2,0.0,1.1543,--|--|--,3,disagreement in flag. doublet partially on sky...
516,35185754053284109,-1,0.33.0.dev3890,0,GALAXY,1.1543,BLH,3.0,--,--,...,85.516308,1.1543,GALAXY,3.0,2,0.0,1.1543,--|--|--,3,disagreement in flag. doublet partially on sky...
566,35185754053284109,-1,0.33.0.dev3890,0,GALAXY,1.1543,TWL,3.0,--,--,...,85.516308,1.1543,GALAXY,3.0,2,0.0,1.1543,--|--|--,3,disagreement in flag. doublet partially on sky...


conflict ID =  25


In [145]:
#copy this text to a new cell to display the conflict
conflict_id=26
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
483,35185754057477287,QSO,2.0301,DRP,4.0,--,2.0401,--,--,2.0401,QSO,4.0,0,2.0301,0.0033,--|--|--,3,none
533,35185754057477287,QSO,2.0301,BLH,4.0,--,--,--,--,2.0301,QSO,4.0,0,2.0301,0.0033,--|--|--,3,none
583,35185754057477287,QSO,2.0301,TWL,4.0,--,--,--,--,2.0301,QSO,4.0,0,2.0301,0.0033,--|--|--,3,none


In [146]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 2.0401
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'slight adjustment to redrock z. take VI z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
483,35185754057477287,-1,0.33.0.dev3890,0,QSO,2.0301,DRP,4.0,--,2.0401,...,288.39503,2.0401,QSO,4.0,0,0.0033,2.0301,--|--|--,3,slight adjustment to redrock z. take VI z.
533,35185754057477287,-1,0.33.0.dev3890,0,QSO,2.0301,BLH,4.0,--,--,...,288.39503,2.0401,QSO,4.0,0,0.0033,2.0301,--|--|--,3,slight adjustment to redrock z. take VI z.
583,35185754057477287,-1,0.33.0.dev3890,0,QSO,2.0301,TWL,4.0,--,--,...,288.39503,2.0401,QSO,4.0,0,0.0033,2.0301,--|--|--,3,slight adjustment to redrock z. take VI z.


conflict ID =  26


In [149]:
#copy this text to a new cell to display the conflict
conflict_id=27
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
487,35185754057477995,GALAXY,1.3242,DRP,3.0,--,--,--,--,1.3242,GALAXY,2.0,3,1.3242,0.0,--|--| Model fits sky residuals,3,none
537,35185754057477995,GALAXY,1.3242,BLH,3.0,--,--,--,--,1.3242,GALAXY,2.0,3,1.3242,0.0,--|--| Model fits sky residuals,3,none
587,35185754057477995,GALAXY,1.3242,TWL,0.0,--,--,--,Model fits sky residuals,1.3242,GALAXY,2.0,3,1.3242,0.0,--|--| Model fits sky residuals,3,none


In [150]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. fit is not secure, set to mean flag of 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
487,35185754057477995,-1,0.33.0.dev3890,0,GALAXY,1.3242,DRP,2.0,--,--,...,72.497853,1.3242,GALAXY,2.0,3,0.0,1.3242,--|--| Model fits sky residuals,3,"disagreement in flag. fit is not secure, set t..."
537,35185754057477995,-1,0.33.0.dev3890,0,GALAXY,1.3242,BLH,2.0,--,--,...,72.497853,1.3242,GALAXY,2.0,3,0.0,1.3242,--|--| Model fits sky residuals,3,"disagreement in flag. fit is not secure, set t..."
587,35185754057477995,-1,0.33.0.dev3890,0,GALAXY,1.3242,TWL,2.0,--,--,...,72.497853,1.3242,GALAXY,2.0,3,0.0,1.3242,--|--| Model fits sky residuals,3,"disagreement in flag. fit is not secure, set t..."


conflict ID =  27


In [153]:
#copy this text to a new cell to display the conflict
conflict_id=28
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
492,35185754057478517,GALAXY,1.6056,DRP,1.0,--,--,--,--,1.6056,GALAXY,2.0,2,1.6056,0.0,--|--| Possible [OII] next to sky lines,3,none
542,35185754057478517,GALAXY,1.6056,BLH,2.0,--,--,--,--,1.6056,GALAXY,2.0,2,1.6056,0.0,--|--| Possible [OII] next to sky lines,3,none
592,35185754057478517,GALAXY,1.6056,TWL,3.0,--,--,--,Possible [OII] next to sky lines,1.6056,GALAXY,2.0,2,1.6056,0.0,--|--| Possible [OII] next to sky lines,3,none


In [155]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, but doublet not secure. set to mean of 2'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
492,35185754057478517,-1,0.33.0.dev3890,0,GALAXY,1.6056,DRP,2.0,--,--,...,36.781978,1.6056,GALAXY,2.0,2,0.0,1.6056,--|--| Possible [OII] next to sky lines,3,"flag disagreement, but doublet not secure. set..."
542,35185754057478517,-1,0.33.0.dev3890,0,GALAXY,1.6056,BLH,2.0,--,--,...,36.781978,1.6056,GALAXY,2.0,2,0.0,1.6056,--|--| Possible [OII] next to sky lines,3,"flag disagreement, but doublet not secure. set..."
592,35185754057478517,-1,0.33.0.dev3890,0,GALAXY,1.6056,TWL,2.0,--,--,...,36.781978,1.6056,GALAXY,2.0,2,0.0,1.6056,--|--| Possible [OII] next to sky lines,3,"flag disagreement, but doublet not secure. set..."


conflict ID =  28


In [158]:
#copy this text to a new cell to display the conflict
conflict_id=29
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
493,35185754057478722,GALAXY,1.4611,DRP,2.0,--,--,--,--,1.4611,GALAXY,2.666667,2,1.4611,0.0,--|--| Triple peak [OII] emission,3,none
543,35185754057478722,GALAXY,1.4611,BLH,2.0,--,--,--,--,1.4611,GALAXY,2.666667,2,1.4611,0.0,--|--| Triple peak [OII] emission,3,none
593,35185754057478722,GALAXY,1.4611,TWL,4.0,--,--,--,Triple peak [OII] emission,1.4611,GALAXY,2.666667,2,1.4611,0.0,--|--| Triple peak [OII] emission,3,none


In [159]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.666667
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement due to broadened but not resolved OII. set to mean of 2.7'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
493,35185754057478722,-1,0.33.0.dev3890,0,GALAXY,1.4611,DRP,2.666667,--,--,...,189.383709,1.4611,GALAXY,2.666667,2,0.0,1.4611,--|--| Triple peak [OII] emission,3,flag disagreement due to broadened but not res...
543,35185754057478722,-1,0.33.0.dev3890,0,GALAXY,1.4611,BLH,2.666667,--,--,...,189.383709,1.4611,GALAXY,2.666667,2,0.0,1.4611,--|--| Triple peak [OII] emission,3,flag disagreement due to broadened but not res...
593,35185754057478722,-1,0.33.0.dev3890,0,GALAXY,1.4611,TWL,2.666667,--,--,...,189.383709,1.4611,GALAXY,2.666667,2,0.0,1.4611,--|--| Triple peak [OII] emission,3,flag disagreement due to broadened but not res...


conflict ID =  29


In [162]:
#copy this text to a new cell to display the conflict
conflict_id=30
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
496,35185754061670674,GALAXY,1.4008,DRP,3.0,--,--,--,--,1.4008,GALAXY,3.0,2,1.4008,0.0,--|--|--,3,none
546,35185754061670674,GALAXY,1.4008,BLH,2.0,--,--,--,--,1.4008,GALAXY,3.0,2,1.4008,0.0,--|--|--,3,none
596,35185754061670674,GALAXY,1.4008,TWL,4.0,--,--,--,--,1.4008,GALAXY,3.0,2,1.4008,0.0,--|--|--,3,none


In [163]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'doublet partially on sky line, still secure. set to mean flag 3.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
496,35185754061670674,-1,0.33.0.dev3890,0,GALAXY,1.4008,DRP,3.0,--,--,...,283.147376,1.4008,GALAXY,3.0,2,0.0,1.4008,--|--|--,3,"doublet partially on sky line, still secure. s..."
546,35185754061670674,-1,0.33.0.dev3890,0,GALAXY,1.4008,BLH,3.0,--,--,...,283.147376,1.4008,GALAXY,3.0,2,0.0,1.4008,--|--|--,3,"doublet partially on sky line, still secure. s..."
596,35185754061670674,-1,0.33.0.dev3890,0,GALAXY,1.4008,TWL,3.0,--,--,...,283.147376,1.4008,GALAXY,3.0,2,0.0,1.4008,--|--|--,3,"doublet partially on sky line, still secure. s..."


conflict ID =  30


In [166]:
#copy this text to a new cell to display the conflict
conflict_id=31
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
451,35185754061671838,GALAXY,0.6973,DRP,2.0,--,--,--,--,0.6973,GALAXY,2.666667,2,0.6973,0.0,--|--| Not sure the model is good.,3,none
501,35185754061671838,GALAXY,0.6973,BLH,4.0,--,--,--,--,0.6973,GALAXY,2.666667,2,0.6973,0.0,--|--| Not sure the model is good.,3,none
551,35185754061671838,GALAXY,0.6973,TWL,2.0,--,--,--,Not sure the model is good.,0.6973,GALAXY,2.666667,2,0.6973,0.0,--|--| Not sure the model is good.,3,none


In [167]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2.666667
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, weak OII. take mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
451,35185754061671838,-1,0.33.0.dev3890,0,GALAXY,0.6973,DRP,2.666667,--,--,...,101.941671,0.6973,GALAXY,2.666667,2,0.0,0.6973,--|--| Not sure the model is good.,3,"disagreement in flag, weak OII. take mean flag."
501,35185754061671838,-1,0.33.0.dev3890,0,GALAXY,0.6973,BLH,2.666667,--,--,...,101.941671,0.6973,GALAXY,2.666667,2,0.0,0.6973,--|--| Not sure the model is good.,3,"disagreement in flag, weak OII. take mean flag."
551,35185754061671838,-1,0.33.0.dev3890,0,GALAXY,0.6973,TWL,2.666667,--,--,...,101.941671,0.6973,GALAXY,2.666667,2,0.0,0.6973,--|--| Not sure the model is good.,3,"disagreement in flag, weak OII. take mean flag."


conflict ID =  31


In [170]:
#copy this text to a new cell to display the conflict
conflict_id=32
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
613,35185760059523574,GALAXY,0.6155,TWL,3.0,RS,1.1698,--,Possible broad [OII] emission at z~1.1698; ca...,1.1698,GALAXY,2.333333,2,0.6155,0.343423,Possible broad [OII] emission at z~1.1698; ca...,3,none
663,35185760059523574,GALAXY,0.6155,DRP,1.0,--,--,--,--,0.6155,GALAXY,2.333333,2,0.6155,0.343423,Possible broad [OII] emission at z~1.1698; ca...,3,none
713,35185760059523574,GALAXY,0.6155,BJW,3.0,RS,1.1703,GALAXY,bad continuum; pipeline put [O III] on a line...,1.1703,GALAXY,2.333333,2,0.6155,0.343423,Possible broad [OII] emission at z~1.1698; ca...,3,none


In [171]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = (1.1703 + 1.1698)/2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors suggest same VI z. take mean. OII line broadned but doublet not resolved.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
613,35185760059523574,-1,0.33.0.dev3890,0,GALAXY,0.6155,TWL,3.0,RS,1.1698,...,50.077787,1.17005,GALAXY,2.333333,2,0.343423,0.6155,Possible broad [OII] emission at z~1.1698; ca...,3,2/3 inspectors suggest same VI z. take mean. O...
663,35185760059523574,-1,0.33.0.dev3890,0,GALAXY,0.6155,DRP,3.0,--,--,...,50.077787,1.17005,GALAXY,2.333333,2,0.343423,0.6155,Possible broad [OII] emission at z~1.1698; ca...,3,2/3 inspectors suggest same VI z. take mean. O...
713,35185760059523574,-1,0.33.0.dev3890,0,GALAXY,0.6155,BJW,3.0,RS,1.1703,...,50.077787,1.17005,GALAXY,2.333333,2,0.343423,0.6155,Possible broad [OII] emission at z~1.1698; ca...,3,2/3 inspectors suggest same VI z. take mean. O...


conflict ID =  32


In [174]:
#copy this text to a new cell to display the conflict
conflict_id=33
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
616,35185760059523701,GALAXY,1.4577,TWL,4.0,S,--,--,Calibration problem,1.4577,GALAXY,3.0,2,1.4577,0.0,Calibration problem|--| bad continuum; good [...,3,none
666,35185760059523701,GALAXY,1.4577,DRP,2.0,--,--,--,--,1.4577,GALAXY,3.0,2,1.4577,0.0,Calibration problem|--| bad continuum; good [...,3,none
716,35185760059523701,GALAXY,1.4577,BJW,3.0,--,--,--,bad continuum; good [O II],1.4577,GALAXY,3.0,2,1.4577,0.0,Calibration problem|--| bad continuum; good [...,3,none


In [175]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag but OII secures z'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
616,35185760059523701,-1,0.33.0.dev3890,0,GALAXY,1.4577,TWL,4.0,S,--,...,235.234165,1.4577,GALAXY,3.0,2,0.0,1.4577,Calibration problem|--| bad continuum; good [...,3,disagreement in flag but OII secures z
666,35185760059523701,-1,0.33.0.dev3890,0,GALAXY,1.4577,DRP,4.0,--,--,...,235.234165,1.4577,GALAXY,3.0,2,0.0,1.4577,Calibration problem|--| bad continuum; good [...,3,disagreement in flag but OII secures z
716,35185760059523701,-1,0.33.0.dev3890,0,GALAXY,1.4577,BJW,4.0,--,--,...,235.234165,1.4577,GALAXY,3.0,2,0.0,1.4577,Calibration problem|--| bad continuum; good [...,3,disagreement in flag but OII secures z


conflict ID =  33


In [178]:
#copy this text to a new cell to display the conflict
conflict_id=34
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
618,35185760059524481,GALAXY,1.6533,TWL,3.0,RS,1.2685,--,Possible [OII] at 1.2685; calibration features,1.2685,GALAXY,2.333333,2,1.6533,0.169628,Possible [OII] at 1.2685; calibration feature...,3,none
668,35185760059524481,GALAXY,1.6533,DRP,1.0,--,--,--,--,1.6533,GALAXY,2.333333,2,1.6533,0.169628,Possible [OII] at 1.2685; calibration feature...,3,none
718,35185760059524481,GALAXY,1.6533,BJW,3.0,RS,1.2687,GALAXY,bad continuum; weird inverted model fit put ab...,1.2687,GALAXY,2.333333,2,1.6533,0.169628,Possible [OII] at 1.2685; calibration feature...,3,none


In [179]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2686
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'VI z is correct, flag 3.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
618,35185760059524481,-1,0.33.0.dev3890,0,GALAXY,1.6533,TWL,3.0,RS,1.2685,...,1.947584,1.2686,GALAXY,2.333333,2,0.169628,1.6533,Possible [OII] at 1.2685; calibration feature...,3,"VI z is correct, flag 3."
668,35185760059524481,-1,0.33.0.dev3890,0,GALAXY,1.6533,DRP,3.0,--,--,...,1.947584,1.2686,GALAXY,2.333333,2,0.169628,1.6533,Possible [OII] at 1.2685; calibration feature...,3,"VI z is correct, flag 3."
718,35185760059524481,-1,0.33.0.dev3890,0,GALAXY,1.6533,BJW,3.0,RS,1.2687,...,1.947584,1.2686,GALAXY,2.333333,2,0.169628,1.6533,Possible [OII] at 1.2685; calibration feature...,3,"VI z is correct, flag 3."


conflict ID =  34


In [182]:
#copy this text to a new cell to display the conflict
conflict_id=35
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
622,35185760063717666,GALAXY,0.5804,TWL,4.0,RS,0.8702,--,Calibration features around 5800 A,0.8702,GALAXY,2.333333,3,0.5804,0.183371,Calibration features around 5800 A|--| bad co...,3,none
672,35185760063717666,GALAXY,0.5804,DRP,2.0,--,--,--,--,0.5804,GALAXY,2.333333,3,0.5804,0.183371,Calibration features around 5800 A|--| bad co...,3,none
722,35185760063717666,GALAXY,0.5804,BJW,1.0,S,--,--,bad continuum; absorption line fit is temptin...,0.5804,GALAXY,2.333333,3,0.5804,0.183371,Calibration features around 5800 A|--| bad co...,3,none


In [183]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.8702
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'calibration issues. disagreement in flag and z, but vi z is secure: OII, Hb'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
622,35185760063717666,-1,0.33.0.dev3890,0,GALAXY,0.5804,TWL,3.0,RS,0.8702,...,32.114297,0.8702,GALAXY,2.333333,3,0.183371,0.5804,Calibration features around 5800 A|--| bad co...,3,calibration issues. disagreement in flag and z...
672,35185760063717666,-1,0.33.0.dev3890,0,GALAXY,0.5804,DRP,3.0,--,--,...,32.114297,0.8702,GALAXY,2.333333,3,0.183371,0.5804,Calibration features around 5800 A|--| bad co...,3,calibration issues. disagreement in flag and z...
722,35185760063717666,-1,0.33.0.dev3890,0,GALAXY,0.5804,BJW,3.0,S,--,...,32.114297,0.8702,GALAXY,2.333333,3,0.183371,0.5804,Calibration features around 5800 A|--| bad co...,3,calibration issues. disagreement in flag and z...


conflict ID =  35


In [186]:
#copy this text to a new cell to display the conflict
conflict_id=36
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
624,35185760063718971,GALAXY,0.4469,TWL,3.0,S,1.0158,--,Possible [OII] emission at z~1.0158; calibrat...,1.0158,GALAXY,2.0,3,0.4469,0.393185,Possible [OII] emission at z~1.0158; calibrat...,3,none
674,35185760063718971,GALAXY,0.4469,DRP,0.0,S,--,--,--,0.4469,GALAXY,2.0,3,0.4469,0.393185,Possible [OII] emission at z~1.0158; calibrat...,3,none
724,35185760063718971,GALAXY,0.4469,BJW,3.0,RS,1.0157,--,bad continuum; [O II] at z=1.01 is doublet and...,1.0157,GALAXY,2.0,3,0.4469,0.393185,Possible [OII] emission at z~1.0158; calibrat...,3,none


In [187]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0158
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'reduction issues. 2/3 of inspectors give same VI z based on OII. secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
624,35185760063718971,-1,0.33.0.dev3890,0,GALAXY,0.4469,TWL,3.0,S,1.0158,...,44.694764,1.0158,GALAXY,2.0,3,0.393185,0.4469,Possible [OII] emission at z~1.0158; calibrat...,3,reduction issues. 2/3 of inspectors give same ...
674,35185760063718971,-1,0.33.0.dev3890,0,GALAXY,0.4469,DRP,3.0,S,--,...,44.694764,1.0158,GALAXY,2.0,3,0.393185,0.4469,Possible [OII] emission at z~1.0158; calibrat...,3,reduction issues. 2/3 of inspectors give same ...
724,35185760063718971,-1,0.33.0.dev3890,0,GALAXY,0.4469,BJW,3.0,RS,1.0157,...,44.694764,1.0158,GALAXY,2.0,3,0.393185,0.4469,Possible [OII] emission at z~1.0158; calibrat...,3,reduction issues. 2/3 of inspectors give same ...


conflict ID =  36


In [190]:
#copy this text to a new cell to display the conflict
conflict_id = 37
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
625,35185760063720182,QSO,0.9791,TWL,4.0,S,--,--,Calibration features around 5800 A; broad [OI...,0.9791,QSO,3.0,2,0.979333,0.000354,Calibration features around 5800 A; broad [OI...,3,none
675,35185760063720182,QSO,0.9791,DRP,2.0,S,--,--,--,0.9791,QSO,3.0,2,0.979333,0.000354,Calibration features around 5800 A; broad [OI...,3,none
725,35185760063720182,QSO,0.9791,BJW,3.0,CS,0.9798,GALAXY,bad continuum; ok [O II] but fit with QSO model,0.9798,GALAXY,3.0,2,0.979333,0.000354,Calibration features around 5800 A; broad [OI...,3,none


In [191]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'big reduction issues, but inspectors find OII believable. not a QSO.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
625,35185760063720182,-1,0.33.0.dev3890,0,QSO,0.9791,TWL,3.0,S,--,...,30.619807,0.9791,GALAXY,3.0,2,0.000354,0.979333,Calibration features around 5800 A; broad [OI...,3,"big reduction issues, but inspectors find OII ..."
675,35185760063720182,-1,0.33.0.dev3890,0,QSO,0.9791,DRP,3.0,S,--,...,30.619807,0.9791,GALAXY,3.0,2,0.000354,0.979333,Calibration features around 5800 A; broad [OI...,3,"big reduction issues, but inspectors find OII ..."
725,35185760063720182,-1,0.33.0.dev3890,0,QSO,0.9791,BJW,3.0,CS,0.9798,...,30.619807,0.9798,GALAXY,3.0,2,0.000354,0.979333,Calibration features around 5800 A; broad [OI...,3,"big reduction issues, but inspectors find OII ..."


conflict ID =  37


In [195]:
#copy this text to a new cell to display the conflict
conflict_id=38
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
626,35185760063720299,QSO,0.9698,TWL,4.0,S,--,--,Calibration features around 5800 A; broad [OI...,0.9698,QSO,2.666667,3,0.9698,0.0,Calibration features around 5800 A; broad [OI...,3,none
676,35185760063720299,QSO,0.9698,DRP,1.0,S,--,--,--,0.9698,QSO,2.666667,3,0.9698,0.0,Calibration features around 5800 A; broad [OI...,3,none
726,35185760063720299,QSO,0.9698,BJW,3.0,CS,--,GALAXY,bad continuum; good [O II]; model fit a QSO,0.9698,GALAXY,2.666667,3,0.9698,0.0,Calibration features around 5800 A; broad [OI...,3,none


In [196]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'bas reduction. disagreement in flag. OII compromised by noise, continuum is bad. flag 2. not a QSO.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
626,35185760063720299,-1,0.33.0.dev3890,0,QSO,0.9698,TWL,2.0,S,--,...,11.974498,0.9698,GALAXY,2.666667,3,0.0,0.9698,Calibration features around 5800 A; broad [OI...,3,bas reduction. disagreement in flag. OII compr...
676,35185760063720299,-1,0.33.0.dev3890,0,QSO,0.9698,DRP,2.0,S,--,...,11.974498,0.9698,GALAXY,2.666667,3,0.0,0.9698,Calibration features around 5800 A; broad [OI...,3,bas reduction. disagreement in flag. OII compr...
726,35185760063720299,-1,0.33.0.dev3890,0,QSO,0.9698,BJW,2.0,CS,--,...,11.974498,0.9698,GALAXY,2.666667,3,0.0,0.9698,Calibration features around 5800 A; broad [OI...,3,bas reduction. disagreement in flag. OII compr...


conflict ID =  38


In [199]:
#copy this text to a new cell to display the conflict
conflict_id=39
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
627,35185760063720839,GALAXY,0.446,TWL,3.0,S,1.1010,--,Possible [OII] at z~1.01; calibration feature...,1.1010,GALAXY,2.333333,2,0.446,0.45325,Possible [OII] at z~1.01; calibration feature...,3,none
677,35185760063720839,GALAXY,0.446,DRP,1.0,S,--,--,--,0.4460,GALAXY,2.333333,2,0.446,0.45325,Possible [OII] at z~1.01; calibration feature...,3,none
727,35185760063720839,GALAXY,0.446,BJW,3.0,RS,1.1014,GALAXY,bad continuum; good [O II],1.1014,GALAXY,2.333333,2,0.446,0.45325,Possible [OII] at z~1.01; calibration feature...,3,none


In [200]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = (1.1010 + 1.1014)/2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'calibration issues. disagreement in flag and z, but OII secures VI z'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
627,35185760063720839,-1,0.33.0.dev3890,0,GALAXY,0.446,TWL,3.0,S,1.1010,...,117.687322,1.1012,GALAXY,2.333333,2,0.45325,0.446,Possible [OII] at z~1.01; calibration feature...,3,calibration issues. disagreement in flag and z...
677,35185760063720839,-1,0.33.0.dev3890,0,GALAXY,0.446,DRP,3.0,S,--,...,117.687322,1.1012,GALAXY,2.333333,2,0.45325,0.446,Possible [OII] at z~1.01; calibration feature...,3,calibration issues. disagreement in flag and z...
727,35185760063720839,-1,0.33.0.dev3890,0,GALAXY,0.446,BJW,3.0,RS,1.1014,...,117.687322,1.1012,GALAXY,2.333333,2,0.45325,0.446,Possible [OII] at z~1.01; calibration feature...,3,calibration issues. disagreement in flag and z...


conflict ID =  39


In [203]:
#copy this text to a new cell to display the conflict
conflict_id=40
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
628,35185760063720985,GALAXY,0.6047,TWL,3.0,RS,1.1554,--,Broad [OII] emission at z~1.1554; Calibration ...,1.1554,GALAXY,2.666667,1,0.6047,0.343366,Broad [OII] emission at z~1.1554; Calibration ...,3,none
678,35185760063720985,GALAXY,0.6047,DRP,2.0,--,--,--,--,0.6047,GALAXY,2.666667,1,0.6047,0.343366,Broad [OII] emission at z~1.1554; Calibration ...,3,none
728,35185760063720985,GALAXY,0.6047,BJW,3.0,RS,1.1557,GALAXY,bad continuum; model fit [O III] to line that...,1.1557,GALAXY,2.666667,1,0.6047,0.343366,Broad [OII] emission at z~1.1554; Calibration ...,3,none


In [204]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.1555
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'VI z based on strong OII. broadned not resolved.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
628,35185760063720985,-1,0.33.0.dev3890,0,GALAXY,0.6047,TWL,3.0,RS,1.1554,...,65.893808,1.1555,GALAXY,2.666667,1,0.343366,0.6047,Broad [OII] emission at z~1.1554; Calibration ...,3,VI z based on strong OII. broadned not resolved.
678,35185760063720985,-1,0.33.0.dev3890,0,GALAXY,0.6047,DRP,3.0,--,--,...,65.893808,1.1555,GALAXY,2.666667,1,0.343366,0.6047,Broad [OII] emission at z~1.1554; Calibration ...,3,VI z based on strong OII. broadned not resolved.
728,35185760063720985,-1,0.33.0.dev3890,0,GALAXY,0.6047,BJW,3.0,RS,1.1557,...,65.893808,1.1555,GALAXY,2.666667,1,0.343366,0.6047,Broad [OII] emission at z~1.1554; Calibration ...,3,VI z based on strong OII. broadned not resolved.


conflict ID =  40


In [207]:
#copy this text to a new cell to display the conflict
conflict_id=41
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
602,35185760067913133,GALAXY,0.009,TWL,3.0,R,1.2710,--,Possible Broad [OII] emission at z~1.2710; mod...,1.2710,GALAXY,3.333333,1,0.009,1.250942,Possible Broad [OII] emission at z~1.2710; mod...,3,none
652,35185760067913133,GALAXY,0.009,DRP,4.0,--,--,--,--,0.0090,GALAXY,3.333333,1,0.009,1.250942,Possible Broad [OII] emission at z~1.2710; mod...,3,none
702,35185760067913133,GALAXY,0.009,BJW,3.0,RS,1.2712,GALAXY,continuum jump; inverted model; ok [O II] at ...,1.2712,GALAXY,3.333333,1,0.009,1.250942,Possible Broad [OII] emission at z~1.2710; mod...,3,none


In [208]:
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2711
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors suggest VI z based on OII.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
602,35185760067913133,-1,0.33.0.dev3890,0,GALAXY,0.009,TWL,3.0,R,1.2710,...,787.545547,1.2711,GALAXY,3.333333,1,1.250942,0.009,Possible Broad [OII] emission at z~1.2710; mod...,3,2/3 inspectors suggest VI z based on OII.
652,35185760067913133,-1,0.33.0.dev3890,0,GALAXY,0.009,DRP,3.0,--,--,...,787.545547,1.2711,GALAXY,3.333333,1,1.250942,0.009,Possible Broad [OII] emission at z~1.2710; mod...,3,2/3 inspectors suggest VI z based on OII.
702,35185760067913133,-1,0.33.0.dev3890,0,GALAXY,0.009,BJW,3.0,RS,1.2712,...,787.545547,1.2711,GALAXY,3.333333,1,1.250942,0.009,Possible Broad [OII] emission at z~1.2710; mod...,3,2/3 inspectors suggest VI z based on OII.


conflict ID =  41


In [211]:
#copy this text to a new cell to display the conflict
conflict_id=42
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
635,35185760067913744,GALAXY,0.4364,TWL,0.0,S,--,--,Calibration features around 5800 A,0.4364,GALAXY,1.0,3,0.4364,0.391743,Calibration features around 5800 A|--|bad con...,3,none
685,35185760067913744,GALAXY,0.4364,DRP,0.0,S,--,--,--,0.4364,GALAXY,1.0,3,0.4364,0.391743,Calibration features around 5800 A|--|bad con...,3,none
735,35185760067913744,GALAXY,0.4364,BJW,3.0,RS,0.9991,GALAXY,bad continuum; [O II] doublet,0.9991,GALAXY,1.0,3,0.4364,0.391743,Calibration features around 5800 A|--|bad con...,3,none


In [212]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9991
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'bad calibration. VI z based on the doublet is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
635,35185760067913744,-1,0.33.0.dev3890,0,GALAXY,0.4364,TWL,3.0,S,--,...,7.670298,0.9991,GALAXY,1.0,3,0.391743,0.4364,Calibration features around 5800 A|--|bad con...,3,bad calibration. VI z based on the doublet is ...
685,35185760067913744,-1,0.33.0.dev3890,0,GALAXY,0.4364,DRP,3.0,S,--,...,7.670298,0.9991,GALAXY,1.0,3,0.391743,0.4364,Calibration features around 5800 A|--|bad con...,3,bad calibration. VI z based on the doublet is ...
735,35185760067913744,-1,0.33.0.dev3890,0,GALAXY,0.4364,BJW,3.0,RS,0.9991,...,7.670298,0.9991,GALAXY,1.0,3,0.391743,0.4364,Calibration features around 5800 A|--|bad con...,3,bad calibration. VI z based on the doublet is ...


conflict ID =  42


In [215]:
#copy this text to a new cell to display the conflict
conflict_id=43
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
636,35185760067914129,GALAXY,0.4501,TWL,2.0,RS,1.0947,--,Possible [OII] emission at z~1.0947; Calibrat...,1.0947,GALAXY,1.666667,3,0.4501,0.444521,Possible [OII] emission at z~1.0947; Calibrat...,3,none
686,35185760067914129,GALAXY,0.4501,DRP,0.0,S,--,--,--,0.4501,GALAXY,1.666667,3,0.4501,0.444521,Possible [OII] emission at z~1.0947; Calibrat...,3,none
736,35185760067914129,GALAXY,0.4501,BJW,3.0,RS,1.0946,GALAXY,bad continuum; [O II] doublet,1.0946,GALAXY,1.666667,3,0.4501,0.444521,Possible [OII] emission at z~1.0947; Calibrat...,3,none


In [216]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0946
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'issues in reduction. VI z secured by OII.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
636,35185760067914129,-1,0.33.0.dev3890,0,GALAXY,0.4501,TWL,3.0,RS,1.0947,...,45.130034,1.0946,GALAXY,1.666667,3,0.444521,0.4501,Possible [OII] emission at z~1.0947; Calibrat...,3,issues in reduction. VI z secured by OII.
686,35185760067914129,-1,0.33.0.dev3890,0,GALAXY,0.4501,DRP,3.0,S,--,...,45.130034,1.0946,GALAXY,1.666667,3,0.444521,0.4501,Possible [OII] emission at z~1.0947; Calibrat...,3,issues in reduction. VI z secured by OII.
736,35185760067914129,-1,0.33.0.dev3890,0,GALAXY,0.4501,BJW,3.0,RS,1.0946,...,45.130034,1.0946,GALAXY,1.666667,3,0.444521,0.4501,Possible [OII] emission at z~1.0947; Calibrat...,3,issues in reduction. VI z secured by OII.


conflict ID =  43


In [219]:
#copy this text to a new cell to display the conflict
conflict_id=44
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
637,35185760067914335,GALAXY,1.6639,TWL,4.0,RS,1.2782,--,[OII] at z~1.2782; Calibration features aroun...,1.2782,GALAXY,3.0,2,1.6639,0.1693,[OII] at z~1.2782; Calibration features aroun...,3,none
687,35185760067914335,GALAXY,1.6639,DRP,2.0,S,--,--,--,1.6639,GALAXY,3.0,2,1.6639,0.1693,[OII] at z~1.2782; Calibration features aroun...,3,none
737,35185760067914335,GALAXY,1.6639,BJW,3.0,RS,1.2782,GALAXY,bad continuum; [O II] doublet,1.2782,GALAXY,3.0,2,1.6639,0.1693,[OII] at z~1.2782; Calibration features aroun...,3,none


In [220]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.2782
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. OII doublet secures VI z'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
637,35185760067914335,-1,0.33.0.dev3890,0,GALAXY,1.6639,TWL,3.0,RS,1.2782,...,104.281946,1.2782,GALAXY,3.0,2,0.1693,1.6639,[OII] at z~1.2782; Calibration features aroun...,3,disagreement in z and flag. OII doublet secure...
687,35185760067914335,-1,0.33.0.dev3890,0,GALAXY,1.6639,DRP,3.0,S,--,...,104.281946,1.2782,GALAXY,3.0,2,0.1693,1.6639,[OII] at z~1.2782; Calibration features aroun...,3,disagreement in z and flag. OII doublet secure...
737,35185760067914335,-1,0.33.0.dev3890,0,GALAXY,1.6639,BJW,3.0,RS,1.2782,...,104.281946,1.2782,GALAXY,3.0,2,0.1693,1.6639,[OII] at z~1.2782; Calibration features aroun...,3,disagreement in z and flag. OII doublet secure...


conflict ID =  44


In [223]:
#copy this text to a new cell to display the conflict
conflict_id= 45
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
638,35185760067914470,GALAXY,0.7529,TWL,4.0,S,--,--,Calibration features around 5800 A,0.7529,GALAXY,3.0,2,0.7529,0.0,Calibration features around 5800 A|--| bad co...,3,none
688,35185760067914470,GALAXY,0.7529,DRP,2.0,S,--,--,--,0.7529,GALAXY,3.0,2,0.7529,0.0,Calibration features around 5800 A|--| bad co...,3,none
738,35185760067914470,GALAXY,0.7529,BJW,3.0,S,--,--,bad continuum; ok [O II] and poss Hbeta,0.7529,GALAXY,3.0,2,0.7529,0.0,Calibration features around 5800 A|--| bad co...,3,none


In [224]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. terrible spectrum, but take flag of 3 based on OII'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
638,35185760067914470,-1,0.33.0.dev3890,0,GALAXY,0.7529,TWL,3.0,S,--,...,3.205188,0.7529,GALAXY,3.0,2,0.0,0.7529,Calibration features around 5800 A|--| bad co...,3,"disagreement in flag. terrible spectrum, but t..."
688,35185760067914470,-1,0.33.0.dev3890,0,GALAXY,0.7529,DRP,3.0,S,--,...,3.205188,0.7529,GALAXY,3.0,2,0.0,0.7529,Calibration features around 5800 A|--| bad co...,3,"disagreement in flag. terrible spectrum, but t..."
738,35185760067914470,-1,0.33.0.dev3890,0,GALAXY,0.7529,BJW,3.0,S,--,...,3.205188,0.7529,GALAXY,3.0,2,0.0,0.7529,Calibration features around 5800 A|--| bad co...,3,"disagreement in flag. terrible spectrum, but t..."


conflict ID =  45


In [227]:
#copy this text to a new cell to display the conflict
conflict_id= 46
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
640,35185760067915063,GALAXY,0.4061,TWL,4.0,RS,1.4756,--,[OII] at z~1.4756; calibration features 5800 A,1.4756,GALAXY,2.666667,3,0.4061,0.760614,[OII] at z~1.4756; calibration features 5800 ...,3,none
690,35185760067915063,GALAXY,0.4061,DRP,1.0,S,--,--,--,0.4061,GALAXY,2.666667,3,0.4061,0.760614,[OII] at z~1.4756; calibration features 5800 ...,3,none
740,35185760067915063,GALAXY,0.4061,BJW,3.0,RS,1.4754,GALAXY,bad continuum; model misidentified [O II] dou...,1.4754,GALAXY,2.666667,3,0.4061,0.760614,[OII] at z~1.4756; calibration features 5800 ...,3,none


In [228]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.4755
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'issues in reduction. disagreement in flag, but doublet secures z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
640,35185760067915063,-1,0.33.0.dev3890,0,GALAXY,0.4061,TWL,4.0,RS,1.4756,...,82.674709,1.4755,GALAXY,2.666667,3,0.760614,0.4061,[OII] at z~1.4756; calibration features 5800 ...,3,"issues in reduction. disagreement in flag, but..."
690,35185760067915063,-1,0.33.0.dev3890,0,GALAXY,0.4061,DRP,4.0,S,--,...,82.674709,1.4755,GALAXY,2.666667,3,0.760614,0.4061,[OII] at z~1.4756; calibration features 5800 ...,3,"issues in reduction. disagreement in flag, but..."
740,35185760067915063,-1,0.33.0.dev3890,0,GALAXY,0.4061,BJW,4.0,RS,1.4754,...,82.674709,1.4755,GALAXY,2.666667,3,0.760614,0.4061,[OII] at z~1.4756; calibration features 5800 ...,3,"issues in reduction. disagreement in flag, but..."


conflict ID =  46


In [231]:
#copy this text to a new cell to display the conflict
conflict_id=47
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
641,35185760067915894,GALAXY,1.4569,TWL,4.0,--,--,--,Calibration features around 5800 A,1.4569,GALAXY,3.0,2,1.4569,0.0,Calibration features around 5800 A|--| bad co...,3,none
691,35185760067915894,GALAXY,1.4569,DRP,2.0,S,--,--,--,1.4569,GALAXY,3.0,2,1.4569,0.0,Calibration features around 5800 A|--| bad co...,3,none
741,35185760067915894,GALAXY,1.4569,BJW,3.0,S,--,--,bad continuum; [O II] doublet,1.4569,GALAXY,3.0,2,1.4569,0.0,Calibration features around 5800 A|--| bad co...,3,none


In [232]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'reduction issues. disagreement in flag but doublet secures z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
641,35185760067915894,-1,0.33.0.dev3890,0,GALAXY,1.4569,TWL,4.0,--,--,...,333.179094,1.4569,GALAXY,3.0,2,0.0,1.4569,Calibration features around 5800 A|--| bad co...,3,reduction issues. disagreement in flag but dou...
691,35185760067915894,-1,0.33.0.dev3890,0,GALAXY,1.4569,DRP,4.0,S,--,...,333.179094,1.4569,GALAXY,3.0,2,0.0,1.4569,Calibration features around 5800 A|--| bad co...,3,reduction issues. disagreement in flag but dou...
741,35185760067915894,-1,0.33.0.dev3890,0,GALAXY,1.4569,BJW,4.0,S,--,...,333.179094,1.4569,GALAXY,3.0,2,0.0,1.4569,Calibration features around 5800 A|--| bad co...,3,reduction issues. disagreement in flag but dou...


conflict ID =  47


In [235]:
#copy this text to a new cell to display the conflict
conflict_id=48
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
606,35185760072106551,GALAXY,1.1398,TWL,4.0,--,--,--,[OII] emission on sky lines,1.1398,GALAXY,3.0,2,1.1398,0.0,[OII] emission on sky lines|--|--,3,none
656,35185760072106551,GALAXY,1.1398,DRP,2.0,--,--,--,--,1.1398,GALAXY,3.0,2,1.1398,0.0,[OII] emission on sky lines|--|--,3,none
706,35185760072106551,GALAXY,1.1398,BJW,3.0,--,--,--,--,1.1398,GALAXY,3.0,2,1.1398,0.0,[OII] emission on sky lines|--|--,3,none


In [236]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. OII on top of skyline but secure'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
606,35185760072106551,-1,0.33.0.dev3890,0,GALAXY,1.1398,TWL,3.0,--,--,...,24.186601,1.1398,GALAXY,3.0,2,0.0,1.1398,[OII] emission on sky lines|--|--,3,disagreement in flag. OII on top of skyline bu...
656,35185760072106551,-1,0.33.0.dev3890,0,GALAXY,1.1398,DRP,3.0,--,--,...,24.186601,1.1398,GALAXY,3.0,2,0.0,1.1398,[OII] emission on sky lines|--|--,3,disagreement in flag. OII on top of skyline bu...
706,35185760072106551,-1,0.33.0.dev3890,0,GALAXY,1.1398,BJW,3.0,--,--,...,24.186601,1.1398,GALAXY,3.0,2,0.0,1.1398,[OII] emission on sky lines|--|--,3,disagreement in flag. OII on top of skyline bu...


conflict ID =  48


### and so on...

We should now recompute the conflicts, and not find any.

In [240]:
vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
                       & (x['VI class'].max() > 2)
                       & (len(x) >= 2)) #x is a group by TargetID

In [241]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185754032309199]
Total number of conflicts to resolve:  1


Display anything that was missed (if "Total number of conflicts" isn't zero) and resolve!

In [242]:
#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
21,35185754032309199,GALAXY,1.4716,BJW,3.0,--,--,--,--,1.4716,GALAXY,3.0,2,1.4716,0.0,--|--|--,3,"mismatch in flag, set to mean"
71,35185754032309199,GALAXY,1.4716,CGS,2.0,--,--,--,--,1.4716,GALAXY,3.0,2,1.4716,0.0,--|--|--,3,"mismatch in flag, set to mean"
121,35185754032309199,GALAXY,1.4716,vrk,4.0,--,--,--,--,1.4716,GALAXY,3.0,2,1.4716,0.0,--|--|--,3,"mismatch in flag, set to mean"


In [243]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,dz,vi_combined_z,all VI comments,N_VI,merger comment
21,35185754032309199,-1,0.33.0.dev3890,0,GALAXY,1.4716,BJW,3.0,--,--,...,408.954239,1.4716,GALAXY,3.0,2,0.0,1.4716,--|--|--,3,"mismatch in flag, set to mean"
71,35185754032309199,-1,0.33.0.dev3890,0,GALAXY,1.4716,CGS,3.0,--,--,...,408.954239,1.4716,GALAXY,3.0,2,0.0,1.4716,--|--|--,3,"mismatch in flag, set to mean"
121,35185754032309199,-1,0.33.0.dev3890,0,GALAXY,1.4716,vrk,3.0,--,--,...,408.954239,1.4716,GALAXY,3.0,2,0.0,1.4716,--|--|--,3,"mismatch in flag, set to mean"


conflict ID =  0


Let's check the number of conflicts again.

In [244]:
vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
                       & (x['VI class'].max() > 2)
                       & (len(x) >= 2)) #x is a group by TargetID

In [245]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


## Woohoo!

## Finally
we need to recompute vi_combined_flag and vi_combined_z, after everything is resolved.

In [246]:
vi['vi_combined_flag'] = vi.groupby('TargetID')['VI class'].transform('mean')
vi['vi_combined_z'] = vi.groupby('TargetID')['best redshift'].transform('mean')


## Now we prepare to write to file. 

### The important columns for the truth table construction are **vi_combined_flag** and **vi_combined_z** 

The truth table should **take the redhift value in vi_combined_z** for all objects with **vi_combined_flag >= 2.5** (for discussion, but this catches the cases where there is some disagreement on the 2/3 boundary).

After the merging process, those two columns will be the same for ALL of the members of each group by TargetID, so it doesn't matter which member we write to file. Taking the 1st one just makes this easy. 


In [247]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff', 'dz',
       'vi_combined_z', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')

In [248]:
#vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'all VI comments', 'merger comment', 'N_VI', 'DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+'merged_VI.txt')
vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments', 'merger comment','DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+save_filename)

Check that merged file reads in OK - check comments

In [249]:
merged_file = pd.read_csv(VI_dir+save_filename)

In [250]:
merged_file.keys()

Index(['TargetID', 'Redrock z', 'Redrock spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments',
       'merger comment', 'DELTACHI2', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z',
       'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z'],
      dtype='object')

In [251]:
merged_file

,TargetID,Redrock z,Redrock spectype,best redshift,best spectype,vi_combined_flag,N_VI,all VI comments,merger comment,DELTACHI2,FIBER,FLUX_G,FLUX_R,FLUX_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z
0,35185754028116247,0.5481,GALAXY,0.5481,GALAXY,0.333333,3,no real features|--| fit not convincing (doub...,none,6.121645,846,0.592657,0.784296,1.518387,0.190289,0.251820,0.487521
1,35185754028116361,0.8179,GALAXY,0.8179,GALAXY,4.000000,3,--|--|--,none,528.739231,753,0.488526,1.075064,2.871854,0.329701,0.725549,1.938184
2,35185754028116626,1.2095,GALAXY,1.2095,GALAXY,2.000000,3,Missing data and noise in middle. [O II] may b...,none,43.749015,970,0.398110,0.364910,1.306789,0.180468,0.165418,0.592383
3,35185754028116942,0.8915,GALAXY,0.8915,GALAXY,4.000000,3,--|--|--,none,1331.826093,791,0.675060,0.960820,2.158175,0.229620,0.326821,0.734099
4,35185754028117502,1.0139,GALAXY,1.0139,GALAXY,3.666667,3,--|--|--,none,383.423929,751,0.331463,0.488661,1.278014,0.240064,0.353916,0.925611
5,35185754028117606,0.7678,GALAXY,0.7678,GALAXY,4.000000,3,--|--|--,none,373.012744,752,0.445223,1.027468,3.064434,0.270089,0.623299,1.858998
6,35185754028117638,1.1293,GALAXY,1.1293,GALAXY,3.666667,3,continuum jump in blue - fit ok|--|--,none,33.110367,980,0.500127,0.660811,1.471567,0.276417,0.365226,0.813327
7,35185754028117827,1.1129,GALAXY,1.1129,GALAXY,4.000000,3,--|--|--,none,358.030885,794,0.530768,0.426952,0.960159,0.333779,0.268494,0.603808
8,35185754028118045,0.2120,GALAXY,0.2120,GALAXY,4.000000,3,--|--|--,none,4438.298532,762,0.431211,0.489021,0.391408,0.335715,0.380722,0.304727
9,35185754028118049,1.5280,GALAXY,1.5280,GALAXY,1.000000,3,[O II] among skylines not convincing|--| doubl...,none,19.545464,780,0.236458,0.273540,0.238727,0.184218,0.213108,0.185986


In [252]:
merged_file.loc[9]['merger comment']

'none'

In [253]:
merged_file.loc[9]['all VI comments']

'[O II] among skylines not convincing|--| doublet in skyline noise region + VI redshift gives an equally unsecure doublet'

In [254]:
VI_dir

'/global/homes/r/rtojeiro/SV/VI_files/SV0/ELG/'